##### Load Package

In [3]:
from langchain_openai import ChatOpenAI

from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.prompts import ChatPromptTemplate,FewShotChatMessagePromptTemplate
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain_community.vectorstores import Chroma
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_openai import OpenAI

import os
from dotenv import load_dotenv

##### Set Up Environment 

In [4]:
load_dotenv(dotenv_path = "../Key/.env")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# what Is Example Selector 

* we want to show llm what we want it to do \ 
  one way is to provide examples of how to response in different scenarios \
  however if we have a large list of examples we do not want to throw all those into the prompt \
  some may not fit some may not be relevant \
  so we want to select from a series of examples that allow us to dynamically place in-context information into prompt


# Incorporate An Example Selector In FewShotPromptTemplate

## Initialize Engines

In [4]:
llm = OpenAI()
embeddings = OpenAIEmbeddings()

## Build An Example Selector

##### Use SemanticSimilarityExampleSelector.from_examples()

In [29]:
# Provide Examples Of A Pair Of Animal And The Place Where This Animial Is Usually Found
examples = [
    {"input": "pirate", "output": "ship"},
    {"input": "pilot", "output": "plane"},
    {"input": "driver", "output": "car"},
    {"input": "bird", "output": "nest"},
    {"input": "dog", "output": "gargen"}
]

example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    embeddings,
    FAISS,
    k = 2
    # * k is the number of examples to select
)

print(example_selector.select_examples({"input": "student"}))
print(example_selector.select_examples({"input": "duck"}))

[{'input': 'driver', 'output': 'car'}, {'input': 'pilot', 'output': 'plane'}]
[{'input': 'bird', 'output': 'nest'}, {'input': 'dog', 'output': 'gargen'}]


##### Use SemanticSimilarityExampleSelector

In [36]:
# Provide Examples Of A Pair Of Animal And The Place Where This Animial Is Usually Found
examples = [
    {"input": "pirate", "output": "ship"},
    {"input": "pilot", "output": "plane"},
    {"input": "driver", "output": "car"},
    {"input": "bird", "output": "nest"},
    {"input": "dog", "output": "gargen"}
]


to_vectorize = [" ".join(example.values()) for example in examples]
vectorstore = FAISS.from_texts(to_vectorize, embeddings, metadatas=examples)

example_selector = SemanticSimilarityExampleSelector(
    vectorstore = vectorstore,
    k = 2
)

print(to_vectorize)
print(example_selector.select_examples({"input": "student"}))

['pirate ship', 'pilot plane', 'driver car', 'bird nest', 'dog gargen']
[{'input': 'driver', 'output': 'car'}, {'input': 'pilot', 'output': 'plane'}]


## Add Example Selector In FewShotPromptTemplate

In [33]:
example_prompt = PromptTemplate(
    template = """
    example input: {input}
    example output: {output}
    """,
    input_variables = ["input", "output"]
)


prompt_template = FewShotPromptTemplate(
    example_selector = example_selector,
    example_prompt = example_prompt,

    # Add The Top And Bottom Of The Prompt
    prefix = "give the location an item is usually found in",
    suffix = """
    input: {noun}
    output: 
    """,

    input_variables = ["noun"]
)


prompt = prompt_template.format(noun = "student")

print(prompt)


give the location an item is usually found in


    example input: driver
    example output: car
    


    example input: pilot
    example output: plane
    


    input: student
    output: 
    


In [26]:
llm.invoke(prompt)

' classroom'

# Use FewShotChatMessagePromptTemplate In Chat Model

In [5]:
chat = ChatOpenAI()

In [12]:
examples = [
    {"input": "pirate", "output": "ship"},
    {"input": "pilot", "output": "plane"},
    {"input": "driver", "output": "car"},
    {"input": "bird", "output": "nest"},
    {"input": "dog", "output": "gargen"}
]



example_prompt=ChatPromptTemplate.from_messages(
    [("human", "{input}"),
     ("ai", "{output}")
    ]
)


example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(),
    FAISS,
    k = 2
)


few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_selector = example_selector,
    example_prompt = example_prompt
)


prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "give the location an item is usually found in"),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

prompt = prompt_template.format(input = "student")


print(prompt)
response = chat.invoke(prompt)
print(response)

System: give the location an item is usually found in
Human: driver
AI: car
Human: pilot
AI: plane
Human: student
content='AI: classroom' response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 38, 'total_tokens': 41}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-7c381c75-1497-491c-970f-964a4ffa76a5-0'
